## outline

- 两者底层调的都是
    - `Llama` 类对象的模型结构显然是一致的；
        ```
        tokenizer = Tokenizer(model_path=tokenizer_path)
        model = Transformer(model_args)
        model.load_state_dict(checkpoint, strict=False)
        llama = Llama(model, tokenizer)
        ```
    - `Llama.generate(prompt_tokens, max_gen_len, ...)`
- 差异
    - model weights 不同；
    - prompt_tokens 的组织不同

In [3]:
# tokenizer
from sentencepiece import SentencePieceProcessor
sp_model = SentencePieceProcessor(model_file='./tokenizer.model')

In [7]:
sp_model.bos_id()

1

In [8]:
sp_model.eos_id()

2

In [9]:
sp_model.pad_id()

-1

## 接口层面

```
def text_completion(
    self,
    prompts: List[str],
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_gen_len: Optional[int] = None,
    logprobs: bool = False,
    echo: bool = False,
) -> List[CompletionPrediction]:


def chat_completion(
    self,
    dialogs: List[Dialog],
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_gen_len: Optional[int] = None,
    logprobs: bool = False,
) -> List[ChatPrediction]:
```

## chat 接口抽象

```
class Message(TypedDict):
    role: Role
    content: str


Dialog = List[Message]

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

SPECIAL_TAGS = [B_INST, E_INST, "<<SYS>>", "<</SYS>>"]
UNSAFE_ERROR = "Error: special tags are not allowed as part of the prompt."

```

- role
    - system
    - user
    - assistant
    - "starting with 'system', then 'user' and alternating (u/a/u/a/u...)"
- unsafe requests
    - dialog content 中包含 `"[INST]", "[/INST]", "<<SYS>>\n", "\n<</SYS>>\n\n"`

In [1]:
print("\n<</SYS>>\n\n")


<</SYS>>




### special tags

In [13]:
sp_model.encode('[INST]')

[518, 25580, 29962]

In [14]:
sp_model.encode('what')

[825]

In [23]:
sp_model.encode('[/INST]')

[518, 29914, 25580, 29962]

In [5]:
# '[INST] what is the recipe of mayonnaise? [/INST]'
sp_model.encode('[INST] what is the recipe of mayonnaise? [/INST]')

[518,
 25580,
 29962,
 825,
 338,
 278,
 9522,
 412,
 310,
 1122,
 11586,
 895,
 29973,
 518,
 29914,
 25580,
 29962]